# Mocap Data Development

Specifically a **subset** of the data used in D Holden's DLFCMS project, specifically **CMU Locomotion** categorized data.

This is after the pre-processing in python.

In [1]:
using LinearAlgebra, Statistics, Random
using Quaternions    # For manipulating 3D Geometry
using MeshCat        # For web visualisation / animation
using PyPlot         # Plotting
using AxUtil, Flux   # Optimisation

# small utils libraries
using ProgressMeter, Formatting, ArgCheck
using NPZ

# global utils
squeeze(x) = dropdims(x, dims=tuple(findall(size(x) .== 1)...))  # may be an official fn, but not sure where
unsqueeze(xs, dim) = reshape(xs, (size(xs)[1:dim-1]..., 1, size(xs)[dim:end]...))  # <= from Flux.jl
rowmaj_reshape_3d(x, ix, iy, iz) = (x=reshape(x, ix, iz, iy); permutedims(x, [1,3,2]);)

DIR_MOCAP_MTDS = "../../../mocap-mtds/";   # different cos I'm in dev folder

# MeshCat skeleton visualisation tools
include(joinpath(DIR_MOCAP_MTDS, "mocap_viz.jl"))

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-8356415965691090002\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-10558696560635704151\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio/main\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n        console.warn(\"Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).\");\n        require([\"/assetserver/5794a0bf2e6325e42dc09c7369026176a04edd3e-dist/main.js\"], function (webIOModule) {\n            webIOModule.load_ipython_extension();\n        });\n        warning_div.innerHTML = \"<strong>Loading WebIO Jupyter extension on an ad-hoc basis. Consider enabling the WebIO nbextension for a stabler experience (this should happen automatically when building WebIO).</strong>\";\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<div\n    id=\"webio-warning-8356415965691090002\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    <!-- TODO: link to installation docs. -->\n</div>\n")

Main.mocapviz

In [5]:
# Python interface to reuse Dan Holden's code
using PyCall

# add to PATH
map(["", joinpath(DIR_MOCAP_MTDS, "pyfiles")]) do p
    pushfirst!(PyVector(pyimport("sys")."path"), p); end

# imports
BVHpy = pyimport("BVH");
Animpy = pyimport("Animation");
Quatpy = pyimport("Quaternions");
Pivotspy = pyimport("Pivots")
filterspy = pyimport("scipy.ndimage.filters")   # note scipy not dl by default by Julia

PyObject <module 'scipy.ndimage.filters' from '/Users/alexbird/.julia/conda/3/lib/python3.7/site-packages/scipy/ndimage/filters.py'>

In [6]:
# PyCall Utility Functions
toArray(x) = convert(Array, x)   # Quatpy class operations return Python objects (numpy). Forces convert.
collapseDim3Jl(x) = reshape(x, size(x, 1), :)
collapseDim3Npy(x) = (x=permutedims(x, [1,3,2]); reshape(x, size(x, 1), :);)

# class method __getitem__ def in Quaternions.py doesn't play nicely. Let's write our own:
QuatpyGetItem(pyo, ixs...) = Quatpy.Quaternions(pyo.qs[(ixs..., :, :, :)[1:ndims(pyo.qs)]...])

QuatpyGetItem (generic function with 1 method)

-------------------
## Extract relevant details from BVH file

* Rotational velocity about root [1],
* Global velocity of root in $x, z$ directions [2,3],
* Foot contact points (L,R,heel,toe) [4,5,6,7],
* Local joint positions (rel. to root) x21 x3 [8-70],
* Local joint velocities (rel. to root) x21 x3 [71-133],
* Local joint rotations (rel. to root?) x21 x3 [134-196],
* Forward direction (note $\perp$ to $y$, ie. $y=0$) [197-199]

In [165]:
function process_file(filename)
    
    anim, names, frametime = BVHpy.load(filename)
    
    # Subsample to 60 fps
    anim = get(anim,  range(0, length(anim)-1, step=2))
    
    # Do FK
    global_xforms = Animpy.transforms_global(anim)  # intermediate
    global_positions = global_xforms[:,:,1:3,4] ./ global_xforms[:,:,4:end,4]
    global_rotations = Quatpy.Quaternions.from_transforms(global_xforms)
    

    # Remove Uneeded Joints
    used_joints = [0, 2,  3,  4,  5, 7,  8,  9, 10, 12, 13, 15, 16, 18, 19, 20, 22, 25, 26, 27, 29] .+ 1
         
    positions = global_positions[:, used_joints,:]
    global_rotations = QuatpyGetItem(global_rotations,:,used_joints,:) 
    N = size(positions, 1)
    # ________________________________________________________

    # Put on Floor
    positions[:,:,2] .-= minimum(positions[:,:,2])
    
    # Get Foot Contacts
    velfactor, heightfactor = [0.05,0.05], [3.0, 2.0]
    fid_l, fid_r = [3,4] .+1, [7,8] .+1
    
    feet_l_vmag_sq = sum(x->x^2, diff(positions[:, fid_l, 1:3], dims=1), dims=3) |> squeeze
    feet_l_h = positions[1:end-1, fid_l, 2]
    feet_l = (feet_l_vmag_sq .< velfactor') .& (feet_l_h .< heightfactor')
    
    feet_r_vmag_sq = sum(x->x^2, diff(positions[:, fid_r, 1:3], dims=1), dims=3) |> squeeze
    feet_r_h = positions[1:end-1, fid_r, 2]
    feet_r = (feet_r_vmag_sq .< velfactor') .& (feet_r_h .< heightfactor')
    
    # Get Root Velocity
    velocity = diff(positions[:,1:1,:], dims=1)
    
    # Remove translation
    positions[:,:,1] .-= positions[:,1:1,1]
    positions[:,:,3] .-= positions[:,1:1,3]
    
    # Get Forward Direction
    sdr_l, sdr_r, hip_l, hip_r = 14, 18, 2, 6  #13, 17, 1, 5
    across1 = positions[:,hip_l,:] - positions[:,hip_r,:]
    across0 = positions[:,sdr_l,:] - positions[:,sdr_r,:]
    across = across0 + across1
    across = across ./ sqrt.(sum(x->x^2, across, dims=2))
    
    direction_filterwidth = 20
#     forward = [cross(view(across, i,:), [0., 1, 0])' for i in 1:N] |>  x->reduce(vcat, x)  # crossprod
    forward = hcat(-across[:,3], zeros(size(across, 1), 1), across[:,1])  # crossprod (easy as spec. case)
    forward = filterspy.gaussian_filter1d(forward, direction_filterwidth, axis=0, mode="nearest")
    forward = forward ./ sqrt.(sum(x->x^2, forward, dims=2))
    
    # Get Root Rotation
    target = repeat([0,0,1]', N, 1)
    root_rotation = Quatpy.Quaternions.between(forward, target)
    root_rotation.qs = unsqueeze(root_rotation.qs, 2);
    root_rot_omitlast = QuatpyGetItem(root_rotation, 1:(N-1)) 
    rvelocity = (QuatpyGetItem(root_rotation, 2:N) * -root_rot_omitlast).to_pivots()
    
    # Local Space  # NEW: define position of joints relative to root
    local_positions = positions  # copy(positions)
    local_positions[:,:,1] .-= local_positions[:,1:1,1]  # x rel to root x
    local_positions[:,:,3] .-= local_positions[:,1:1,3]  # z rel to root z
    
    local_positions = root_rot_omitlast * local_positions[1:end-1,:,:]  |> toArray # remove Y rotation from pos
    local_velocities = diff(local_positions, dims=1)
    local_rotations = abs((root_rot_omitlast * QuatpyGetItem(global_rotations, 1:(N-1)))).log()
    
    root_rvelocity = Pivotspy.Pivots.from_quaternions(QuatpyGetItem(root_rotation, 2:N) * -root_rot_omitlast).ps
    global_velocities = root_rot_omitlast * velocity    |> toArray                # remove Y rotation from vel
    
    
    @assert (size(global_velocities, 2) == 1) "output assumes global_velocities dim2 = 1."
    omit_end = 1:(N-2)
    out = hcat(root_rvelocity[omit_end,:],
                global_velocities[omit_end,1,1],
                global_velocities[omit_end,1,3],
                feet_l[omit_end,:], feet_r[omit_end,:],
                collapseDim3Npy(local_positions[omit_end,:,:]),
                collapseDim3Npy(local_velocities),
                collapseDim3Npy(local_rotations[omit_end,:,:]),
                forward[omit_end,:]
        )
    return out
end

process_file (generic function with 1 method)

#### Test that the function is bug-free

Test vs. Dan Holden's code (which I've lightly modified) on all CMU locomotion instances.

In [ ]:

testpy = pyimport("tmptst");
using DelimitedFiles

cmu_loco = readdlm("../cmu/cmu_locomotion_lkp.txt", '\t')[:,1];
database = "cmu"
files = [joinpath("cmu", f * ".bvh") for f in cmu_loco]
files = collect(filter(x-> isfile(x) && x !== "rest.bvh", files));

for (i, f) in enumerate(files)
    tmppy = testpy.process_file(f)
    tmp = process_file(f)
    printfmtln("{:02d}, lines={:04d}, maximum diff = {:.5e}", i, size(tmp,1), maximum(abs.(tmp - tmppy)))
end

--------------

In [166]:
proc = process_file("./cmu/09_12.bvh");

--------------
# Reconstruct global movement

In [393]:
# 
qimag = Quaternions.imag
quat_list(x) = [quat(x[i,:]) for i in 1:size(x,1)]
quat_list_to_mat(x) = reduce(vcat, [qimag(xx)' for xx in x])
quaterion_angle_axis_w_y(θ) = quat(cos(θ/2), 0, sin(θ/2), 0)
apply_rotation(x, qrot) = qrot * x * conj(qrot)


function reconstruct_positions(Y::Matrix, Ymu::Matrix, Ystd::Matrix)
    Y = convert(Matrix{Float64}, Y)   # reduce error propagation from iterative scheme
    Y = Y .* Ystd' .+ Ymu'
    return reconstruct_positions(Y)
end

function reconstruct_positions(Y::Matrix)
    Y = convert(Matrix{Float64}, Y)   # reduce error propagation from iterative scheme
    
    root_r, root_x, root_z, joints = Y[:,1], Y[:,2], Y[:,3], Y[:,8:(63+7)]
    return _joints_fk(joints, root_x, root_z, root_r)
end

function _joints_fk(joints::Matrix{T}, root_x::Vector{T}, root_z::Vector{T}, 
        root_r::Vector{T}) where T <: Number

    n = size(joints, 1)
    joints = rowmaj_reshape_3d(joints, n, 21, 3)
#     joints = reshape(joints, n, 3, 21)
#     joints = permutedims(joints, [1,3,2])
    rotation = Quaternion(1.0)
    offsets = []
    translation = zeros(3)

    for i = 1:n
        joints[i,:,:] = apply_rotation(quat_list(joints[i,:,:]), rotation) |> quat_list_to_mat
        joints[i,:,1] = joints[i,:,1] .+ translation[1]
        joints[i,:,3] = joints[i,:,3] .+ translation[3]
        
        rotation = quaterion_angle_axis_w_y(-root_r[i]) * rotation
        append!(offsets, apply_rotation(quat(0.,0,0,1), rotation))
        translation = translation + qimag(apply_rotation(quat(0., root_x[i], 0, root_z[i]), rotation))
    end
    
    return joints
end

reshape_velocities(Y::Matrix) = rowmaj_reshape_3d(Y[:,(8:(63+7)) .+ 63], size(Y,1), 21, 3)
reshape_rotations(Y::Matrix) = rowmaj_reshape_3d(Y[:,(8:(63+7)) .+ 63], size(Y,1), 21, 3)

reshape_rotations (generic function with 1 method)

In [389]:
recon = reconstruct_positions(proc);

In [ ]:
if !(@isdefined vis) 
    # Create a new visualizer instance (MeshCat.jl)
    vis = Visualizer()
    open(vis)
end
vis = mocapviz.create_animation([recon[270:end,:,:]], "test"; vis=vis, linemesh=[mocapviz.yellowmesh])

--------------
# Construct model inputs/outputs

In [67]:
size(recon)

(957, 21, 3)

In [92]:
ii = 1

1

In [176]:
(1-69:1+59)[9:10:119]

-60:10:50

In [409]:
use_ixs = range(70, stop=size(recon, 1) - 60)
Xs = Matrix{Float32}(undef, length(use_ixs), 48 + 2 + 63)
for (r, ix) in enumerate(use_ixs)
    tix = range(-60, stop=50, step=10)
    traj_pos = recon[ix .+ tix, 1, :]
    traj_pos_full = recon[ix-69:ix+59, :, :]
    
#     forward = proc[ix .+ tix, 1, 197:199]  # note proc not recon
    sdr_l, sdr_r, hip_l, hip_r = 14, 18, 2, 6  #13, 17, 1, 5
    across1 = traj_pos_full[:,hip_l,:] - traj_pos_full[:,hip_r,:]
    across0 = traj_pos_full[:,sdr_l,:] - traj_pos_full[:,sdr_r,:]
    across = across0 + across1
    across = across ./ sqrt.(sum(x->x^2, across, dims=2))
    
    direction_filterwidth = 20
#     forward = [cross(view(across, i,:), [0., 1, 0])' for i in 1:N] |>  x->reduce(vcat, x)  # crossprod
    forward = hcat(-across[:,3], zeros(size(across, 1), 1), across[:,1])  # crossprod (easy as spec. case)
    forward = filterspy.gaussian_filter1d(forward, direction_filterwidth, axis=0, mode="nearest")
    forward = forward ./ sqrt.(sum(x->x^2, forward, dims=2))
    
    Xs[r, 1:24] = vcat(traj_pos[:,1] .- traj_pos[7,1], traj_pos[:,3] .- traj_pos[7,3])
    Xs[r, 25:48] = vcat(forward[9:10:119, 1], forward[9:10:119, 3])
    Xs[r, 49:72] = vcat(proc[ix .+ tix, 197] , proc[ix .+ tix, 199] )
    Xs[r, 73:84] = begin; V=hcat(forward[9:10:119, 1], forward[9:10:119, 3]);  
                    U=hcat(Xs[r, 1:12], Xs[r,13:24]); U ./= sqrt.(sum(x->x^2, U, dims=2)); sum(U .* V, dims=2)[:]; end
end

1. Check if this calc is different from the forward previously calculated.
2. Make secondary extraction/standardisation step where:
    1. trajectory position is diff'd.
    2. trajectory direction is local direction

In [396]:
rots = reshape_rotations(proc);

In [ ]:
function between(u::Vector, v::Vector)
    a = cross(u, v)
    w = norm(u) * norm(v) + u' * v
    return Quaternions(vcat([a ], axis=-1)).normalized()

In [ ]:
_V=hcat(Xs[:,25+6], Xs[:,37+6])

In [ ]:
_U=hcat(diff(Xs[:, 1:12], dims=2)[:,6], diff(Xs[:,13:24], dims=2)[:,6])

In [422]:
_U ./= sqrt.(sum(x->x^2, _U, dims=2));

In [ ]:
plot(sum(_U .* _V, dims=2)[:])

In [430]:
_U

828×2 Array{Float32,2}:
  0.0203199    0.999794
  0.0153396    0.999882
  0.0142091    0.999899
  0.00850111   0.999964
 -0.00406658   0.999992
 -0.00954591   0.999954
 -0.00930803   0.999957
 -0.0233088    0.999728
 -0.0263698    0.999652
 -0.0194437    0.999811
 -0.0163308    0.999867
 -0.0157541    0.999876
 -0.0155494    0.999879
  ⋮                    
 -0.465494    -0.885051
 -0.468268    -0.883586
 -0.467259    -0.88412 
 -0.474417    -0.8803  
 -0.483204    -0.875508
 -0.49045     -0.871469
 -0.501444    -0.86519 
 -0.510004    -0.860172
 -0.521783    -0.853078
 -0.53191     -0.846801
 -0.54479     -0.838572
 -0.55421     -0.832377

In [ ]:
fig, axs = subplots(1,2,figsize=(10,4))
n=size(Xs,1)
tts = ((1:n)./n) .^0.5
axs[1].scatter(Xs[:,25+6] .* tts, Xs[:,37+6] .* tts, c=ColorMap("BuGn")(tts))
_θ = 0.55
_orig = (hcat(Xs[:,49+6], Xs[:,61+6]) * [cos(_θ) -sin(_θ); sin(_θ) cos(_θ)] ) .* tts
axs[2].scatter(_orig[:,1], _orig[:,2], c=ColorMap("BuGn")(tts))
axs[1].set_aspect("equal"); axs[2].set_aspect("equal")

In [ ]:
fig, axs = subplots(1,2,figsize=(10,4))
n=size(Xs,1)
tts = ((1:n)./n) .^0.5
axs[1].scatter(Xs[:,25+6] .* tts, Xs[:,37+6] .* tts, c=ColorMap("BuGn")(tts))
axs[2].scatter(Xs[:,49+6] .* tts, Xs[:,61+6] .* tts, c=ColorMap("BuGn")(tts))
axs[1].set_aspect("equal"); axs[2].set_aspect("equal")

In [426]:
ii

351

In [ ]:
plot(Xs[ii,1:12], Xs[ii,13:24]); gca().set_aspect("equal")
[arrow(Xs[ii,0+j], Xs[ii,12+j], Xs[ii,24+j]*3, Xs[ii,36+j]*3) for j in 1:12]
ii+=10

In [ ]:
fig, axs = subplots(1,2,figsize=(10,4))
n=size(Xs,1)
tts = ((1:n)./n) .^0.5
axs[1].scatter(_V[:,1] .* tts, _V[:,2] .* tts, c=ColorMap("BuGn")(tts))
_θ = 0.
_orig = (hcat(_U[:,1], _U[:,2]) * [cos(_θ) -sin(_θ); sin(_θ) cos(_θ)] ) .* tts
axs[2].scatter(_orig[:,1], _orig[:,2], c=ColorMap("BuGn")(tts))
axs[1].set_aspect("equal"); axs[2].set_aspect("equal")

In [ ]:
[arrow(0, 0, _U[i,1]*0.93, _U[i,2]*0.93, head_width=.02, head_length=0.04, length_includes_head=true, 
           color=ColorMap("cool")(i/(size(_U,1)/10))) for i in 1:10:size(_U,1)];
[text(_U[i,1]*0.93, _U[i,2]*0.93, Int((i-1)/10 + 1)) for i in 1:10:size(_U,1)];
gca().set_ylim(-1,1)
gca().set_xlim(-1,1)

* Derive phase

* standardise, mean/std etc.?